# TensorFlow事始め

# TensorFlowって何？

- http://tensorflow.org/
- [米Google、機械学習システム「TensorFlow」をOSSとして公開](http://itpro.nikkeibp.co.jp/atcl/news/15/111003663/)

# TensorFlowのチュートリアル

## 英語版チュートリアル

- [本家のチュートリアル](http://tensorflow.org/tutorials)が非常に充実している
- 初期リリースということもあり、所々動かないところもあるので注意
- TensorFlowをざっくりと理解するには、次に紹介する日本語のチュートリアル＠Qiitaからやり始めることをオススメします

## 日本語版チュートリアル

- ミニマルな例を扱っている以下のチュートリアル群がいい感じ
    - [TensorFlowのコード分割の考え方](http://qiita.com/sergeant-wizard/items/98ce0993a195475fd7a9)
    - [TensorBoardでスカラのグラフを出力する](http://qiita.com/sergeant-wizard/items/af7a3bd90e786ec982d2)
    - [TensorBoardでデータフローを可視化する](http://qiita.com/sergeant-wizard/items/c98597b8add04b8eea0b)
- 以下の例は上の例をちょこちょこ変更したもの。

In [1]:
# まずtensorflowを読み込む
import tensorflow as tf

In [2]:
# 簡単なデータセットを作成

# 入力データ
x_data = [
  [1., 0., 0.],
  [0., 1., 0.],
  [0., 0., 1.]
]

# 出力データ
y_data = [
  [0., 0., 1.],
  [1., 0., 0.],
  [0., 1., 0.]
]

In [3]:
# 入力データ用のプレースホルダを用意
x_pl = tf.placeholder("float", [None, 3])

In [4]:
# モデルパラメータ（ウエイトとバイアス）を変数として用意
W = tf.Variable(tf.zeros([3, 3]))
b = tf.Variable(tf.zeros([3]))

In [5]:
#  ソフトマックスを用いた多値分類モデルを作成
y_hat = tf.nn.softmax(tf.matmul(x_pl, W) + b)

In [6]:
# 出力データ（ターゲット）用のプレースホルダを用意
y_pl = tf.placeholder("float", [None, 3])

# 損失関数を定義
cross_entropy = - tf.reduce_sum(y_pl * tf.log(y_hat))

In [7]:
# 損失関数を最適化するための手法を用意。ここでは最急勾配法という最適化手法を使用
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [8]:
# 全ての変数を初期化する手順を用意
init = tf.initialize_all_variables()

# プレースホルダに入れる実データを設定
feed_dict={x_pl: x_data, y_pl: y_data}

In [9]:
# セッションを開始
with tf.Session() as sess:
    
    # 初期化を実行
    sess.run(init)
    
    for step in range(1000):
        
        # 学習を１ステップ分実行
        sess.run(train_step, feed_dict=feed_dict)
        
        if step % 100 == 0:
            # 100ステップごとにクロスエントロピーを表示する
            print sess.run(cross_entropy, feed_dict=feed_dict)

3.27587
1.85469
1.17299
0.821553
0.619587
0.492207
0.40589
0.3441
0.297949
0.2623


# Inference, Loss, Trainingにコードを分割

- モデルが複雑になっても、見通しよくコーディングを進めるため

In [10]:
import tensorflow as tf

x_data = [
  [1., 0., 0.],
  [0., 1., 0.],
  [0., 0., 1.]
]

y_data = [
  [0., 0., 1.],
  [1., 0., 0.],
  [0., 1., 0.]
]

In [11]:
# 推定関数を用意：モデルを定義する
def inference_fn(x_pl):
    '''
    Args:
        x_pl: 入力データ用プレースホルダ
        
    Returns:
        モデルによる出力データの予測値
    '''
    
    #  名前空間を設定（TensorBoardが見やすくなる）
    #with tf.name_scope('inference_fn') as scope:

    # モデルパラメータ（ウエイトとバイアス）を変数として用意
    W = tf.Variable(tf.zeros([3, 3]), name='W_var')
    b = tf.Variable(tf.zeros([3]), name='b_var')

    # ソフトマックスを用いた多値分類モデルを作成
    y_hat = tf.nn.softmax(tf.matmul(x_pl, W) + b)
    
    return y_hat

In [12]:
# 誤差関数を用意：
def loss_fn(y_hat, y_pl):
    '''
    Args:
        y_hat:  モデルによる出力データの予測値
        y_pl: 出力データ用プレースホルダ

    Returns:
        クロスエントロピー誤差
    '''
    
    #  名前空間を設定（TensorBoardが見やすくなる）
    #with tf.name_scope('loss_fn') as scope:

    # 損失関数を定義
    cross_entropy = - tf.reduce_sum(y_pl * tf.log(y_hat))

    # TensorBoard用にcross_entropyの値をxentという名前で保存
    tf.scalar_summary("xent", cross_entropy)   # <<<< For TensorBoard
    
    return cross_entropy

In [13]:
def training_fn(loss):
    
    #  名前空間を設定（TensorBoardが見やすくなる）
    #with tf.name_scope('training_fn') as scope:
    
    # 損失関数を最適化するための手法を用意。ここでは最急勾配法という最適化手法を使用
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    return train_step

In [14]:
# TensorFlowにモデルはデフォルトのグラフに組み込まれると伝える
with tf.Graph().as_default():

    # 入力データ用プレースホルダを用意
    x_pl = tf.placeholder("float", [None, 3], name='x_pl')

    # 出力データ用プレースホルダを用意
    y_pl = tf.placeholder("float", [None, 3], name='y_pl')
    
    # プレースホルダに入れる実データを設定
    feed_dict = {x_pl: x_data, y_pl: x_data}
    
    with tf.Session() as sess:

        #モデルの予測値
        y_hat = inference_fn(x_pl)

        # 損失関数
        loss = loss_fn(y_hat, y_pl)

        # 学習
        training_op = training_fn(loss)

        # 変数を初期化する用意
        init = tf.initialize_all_variables()
        # 変数を初期化
        sess.run(init)

        # dataというディレクトリに結果を保存するための用意
        #summary_writer = tf.train.SummaryWriter('logdir') # <<<< For TensorBoard
        #
        # dataというディレクトリに結果と計算グラフを保存するための用意
        summary_writer = tf.train.SummaryWriter('logdir', graph_def=sess.graph_def) # <<<< For TensorBoard
        
        # サマリをまとめる（下に持ってきたらエラー消えた）
        summary_op = tf.merge_all_summaries() # <<<< For TensorBoard
        
        
        for step in range(1000):

            # 学習を１ステップ分実行
            sess.run(training_op, feed_dict=feed_dict)

            if step % 100 == 0:

                # 100ステップごとにクロスエントロピーを表示する
                print sess.run(loss, feed_dict=feed_dict)

                # 結果の保存を実行
                summary_str = sess.run(summary_op, feed_dict=feed_dict) # <<<< For TensorBoard
                summary_writer.add_summary(summary_str, step) # <<<< For TensorBoard

3.27587
1.85469
1.17299
0.821553
0.619587
0.492207
0.40589
0.3441
0.297949
0.2623


In [15]:
!open .

In [ ]:
!tensorboard --logdir=${PWD}/logdir